See [idle_hcc_injection.json](https://gitlab.cern.ch/atlas-itk-strips-daq/itsdaq-sw/-/blob/master/config/triggers/abc_star/idle_hcc_injection.json)

```
{
 "signals": {
     "LCB_1": "0x00000001", "LCB_0": "0x00000002",
     "DatN0_0": "0x00001000", "DatN0_1": "0x00002000",
     "Data1_0": "0x00000100", "Data1_1": "0x00000200",
     "Data0_0": "0x00004000", "Data0_1": "0x00008000",
     "Data2_0": "0x00010000", "Data2_1": "0x00020000",
     "Data3_0": "0x00040000", "Data3_1": "0x00080000",
     "Data4_0": "0x00100000", "Data4_1": "0x00200000",
     "Data5_0": "0x00400000", "Data5_1": "0x00800000",
     "Data6_0": "0x01000000", "Data6_1": "0x02000000",
     "Data7_0": "0x04000000", "Data7_1": "0x08000000",
     "Data8_0": "0x10000000", "Data8_1": "0x20000000",
     "Data9_0": "0x40000000", "Data9_1": "0x80000000",
     "Data10_0":"0x00001000", "Data10_1":"0x00002000"
 },

 "rate": 80,
 "remark": "This is for HCC/vector mode",

  "remark": ["Inject ABC data into HCC",
             "000000000000000000000000000111111000011000000001100001100110",
             "000000011000011000000001111110000000011000000001111110011000",
             "011001100001111110000000011001100001111110000111111000011000",
             "000001100001100110000000011000011110000001111110000000011110",
             "000001111110011000011111100001111111100000011111100001111110"],

 "data": [
   {"par": [
       [
           {
               "raw_hex": {
                   "signals": ["Data2_0", "Data2_1"],
                   "data": "0000000000000 111111000011000000001100001100110 000000011000011000000001111110000000011000000001111110011000 011001100001111110000000011001100001111110000111111000011000 000001100001100110000000011000011110000001111110000000011110 000001111110011000011111100001111111100000011111100001111110"
               }
           }
       ],
       [
           {"label": "LOOP_LCB"},
           {"raw_hex": {"signals": ["LCB_0", "LCB_1"],
                        "data": ["0x7855", "0x7855"]}},
           {"loop": {"to": "LOOP", "count": 500}}
       ]
   ]},

   {"remark": "The end without a comma"}
  ]
}
```

In [1]:
data = '0000000000000 111111000011000000001100001100110 000000011000011000000001111110000000011000000001111110011000 011001100001111110000000011001100001111110000111111000011000 000001100001100110000000011000011110000001111110000000011110 000001111110011000011111100001111111100000011111100001111110'
print(data)

0000000000000 111111000011000000001100001100110 000000011000011000000001111110000000011000000001111110011000 011001100001111110000000011001100001111110000111111000011000 000001100001100110000000011000011110000001111110000000011110 000001111110011000011111100001111111100000011111100001111110


In [2]:
data = data[data.index(" "):].replace(" ", "")
print(data)
print(len(data))

111111000011000000001100001100110000000011000011000000001111110000000011000000001111110011000011001100001111110000000011001100001111110000111111000011000000001100001100110000000011000011110000001111110000000011110000001111110011000011111100001111111100000011111100001111110
273


In [3]:
even = "".join([data[2*i] for i in range(len(data)//2)])
odd = "".join([data[2*i+1] for i in range(len(data)//2)])
print(even)
print(odd)
print(even == odd)

1110010000100101000010010000111000010000111010010100111000010100111001110010000100101000010011000111000011000111010011100111100011100111
1110010000100101000010010000111000010000111010010100111000010100111001110010000100101000010011000111000011000111010011100111100011100111
True


There is an extra "0" after the first packet's stop bit, which I suppose is legal.  The stop bit at the end of the second packet is not doubled, whereas all earlier bits appear twice in this representation.

In [4]:
packet1 = even[:68]
packet2 = even[69:] + "0"
print(packet1)
print(even[68])
print(packet2)

11100100001001010000100100001110000100001110100101001110000101001110
0
11100100001001010000100110001110000110001110100111001111000111001110


In [5]:
from argparse import Namespace
def pbin(b):
    binstr = b
    n = int(binstr,2)
    return "{}/{:x}/{:d}".format(binstr, n, n)
def decode(p):
    """decode an ABC output packet in binary-string format"""
    assert len(p) == 68
    assert p[0:3] == "111"
    assert p[-1] == "0"
    p = p[3:-1]
    assert len(p) == 64
    print(p)
    header = p[:16]
    payload = p[16:]
    typ = header[0:4]
    flag = header[4]
    l0tag = header[5:12]
    assert len(l0tag) == 7
    bcid = header[12:]
    assert len(bcid) == 4
    print("typ={} flag={} l0tag={} bcid={}".
          format(pbin(typ), flag, pbin(l0tag), pbin(bcid)))
    ns = Namespace()
    ns.typ = int(typ,2)
    ns.flag = int(flag,2)
    ns.l0tag = int(l0tag,2)
    ns.bcid = int(bcid,2)
    ns.clusters = 4*[None]
    for i in range(len(ns.clusters)):
        clu = payload[12*i:][:12]
        lc = clu[0]
        addr = clu[1:9]
        assert len(addr) == 8
        patt = clu[9:]
        assert len(patt) == 3
        c = Namespace()
        ns.clusters[i] = c
        c.lc = int(lc,2)
        c.addr = int(addr,2)
        c.patt = int(patt,2)
        print("cluster: lc={} addr={} patt={}".
             format(lc, pbin(addr), pbin(patt)))
    return ns

In [6]:
decode(packet1)

0010000100101000010010000111000010000111010010100111000010100111
typ=0010/2/2 flag=0 l0tag=0010010/12/18 bcid=1000/8/8
cluster: lc=0 addr=10010000/90/144 patt=111/7/7
cluster: lc=0 addr=00010000/10/16 patt=111/7/7
cluster: lc=0 addr=10010100/94/148 patt=111/7/7
cluster: lc=0 addr=00010100/14/20 patt=111/7/7


Namespace(bcid=8, clusters=[Namespace(addr=144, lc=0, patt=7), Namespace(addr=16, lc=0, patt=7), Namespace(addr=148, lc=0, patt=7), Namespace(addr=20, lc=0, patt=7)], flag=0, l0tag=18, typ=2)

In [7]:
decode(packet2)

0010000100101000010011000111000011000111010011100111100011100111
typ=0010/2/2 flag=0 l0tag=0010010/12/18 bcid=1000/8/8
cluster: lc=0 addr=10011000/98/152 patt=111/7/7
cluster: lc=0 addr=00011000/18/24 patt=111/7/7
cluster: lc=0 addr=10011100/9c/156 patt=111/7/7
cluster: lc=1 addr=00011100/1c/28 patt=111/7/7


Namespace(bcid=8, clusters=[Namespace(addr=152, lc=0, patt=7), Namespace(addr=24, lc=0, patt=7), Namespace(addr=156, lc=0, patt=7), Namespace(addr=28, lc=1, patt=7)], flag=0, l0tag=18, typ=2)

In [8]:
run export_to_html --timestamp

<IPython.core.display.Javascript object>